# This ia a test notebook for quantizaiton test - Author: Lujun

## AutoGPTQ

Two method to test and build the structure

   1. Use the already quantized method
   2. Quantized the model by ourself and save it locally

There are also a lot of different implementation framework.

   1. Huggingface
   2. Langchain
   3. Llamaindex

First Method using quantized method

In [1]:
import torch

print("Available GPU devices:", torch.cuda.device_count())
print("Name of the first available GPU:", torch.cuda.get_device_name(0))
print(torch._C._cuda_getCompiledVersion(), "cuda compiled version")
use_cuda = torch.cuda.is_available()

if use_cuda:
    print("__CUDNN VERSION:", torch.backends.cudnn.version())
    print("__Number CUDA Devices:", torch.cuda.device_count())
    print("__CUDA Device Name:", torch.cuda.get_device_name(0))
    print(
        "__CUDA Device Total Memory [GB]:",
        torch.cuda.get_device_properties(0).total_memory / 1e9,
    )

Available GPU devices: 1
Name of the first available GPU: NVIDIA T600 Laptop GPU
12010 cuda compiled version
__CUDNN VERSION: 8902
__Number CUDA Devices: 1
__CUDA Device Name: NVIDIA T600 Laptop GPU
__CUDA Device Total Memory [GB]: 4.294705152


In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Llama-2-7b-Chat-GPTQ", torch_dtype=torch.float16, device_map="auto"
)

In [26]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import GenerationConfig


model_name_or_path = "TheBloke/Llama-2-7B-GPTQ"
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path, device_map="auto", trust_remote_code=True, revision="main"
)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Tell me about AI"
prompt_template = f"""{prompt}

"""
input_ids = tokenizer(prompt_template, return_tensors="pt").input_ids.cuda()


# generation_config = GenerationConfig.from_pretrained("TheBloke/Llama-2-7B-GPTQ")
# generation_config = GenerationConfig.from_pretrained(
#     "./saved_model_config/TheBloke-Llama-2-7B-GPTQ/", "config.json"
# )

ValueError: Found modules on cpu/disk. Using Exllama or Exllamav2 backend requires all the modules to be on GPU.You can deactivate exllama backend by setting `disable_exllama=True` in the quantization config object

In [23]:
prompt = "Instruction: Please strictly answer the question based on the context I gave to you.Context Information: Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.Question: When did Virgin Australia start operating?"
prompt_template = f"""{prompt}

"""
input_ids = tokenizer(prompt_template, return_tensors="pt").input_ids.cuda()

generation_config, unused_kwargs = GenerationConfig.from_pretrained(
    model_name_or_path,
    max_new_tokens=512,
    temperature=0.7,
    return_unused_kwargs=True,
    do_sample=True,
)
output = model.generate(inputs=input_ids, generation_config=generation_config)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Instruction: Please strictly answer the question based on the context I gave to you. Context Information: Lollapalooza /ˌlɒləpəˈluːzə/ (Lolla) is an annual American four-day music festival held in Grant Park in Chicago. It originally started as a touring event in 1991, but several years later, Chicago became its permanent location. Music genres include but are not limited to alternative rock, heavy metal, punk rock, hip hop, and electronic dance music. Lollapalooza has also featured visual arts, nonprofit organizations, and political organizations. The festival, held in Grant Park, hosts an estimated 400,000 people each July and sells out annually. Lollapalooza is one of the largest and most iconic music festivals in the world and one of the longest-running in the United States. Lollapalooza was conceived and created in 1991 as a farewell tour by Perry Farrell, singer of the group Jane's Addiction. Question: Given a reference text about Lollapalooza, where does it take place, who start

In [24]:
prompt = "Instruction: Please strictly answer the question based on the context I gave to you. Context Information: Lollapalooza /ˌlɒləpəˈluːzə/ (Lolla) is an annual American four-day music festival held in Grant Park in Chicago. It originally started as a touring event in 1991, but several years later, Chicago became its permanent location. Music genres include but are not limited to alternative rock, heavy metal, punk rock, hip hop, and electronic dance music. Lollapalooza has also featured visual arts, nonprofit organizations, and political organizations. The festival, held in Grant Park, hosts an estimated 400,000 people each July and sells out annually. Lollapalooza is one of the largest and most iconic music festivals in the world and one of the longest-running in the United States. Lollapalooza was conceived and created in 1991 as a farewell tour by Perry Farrell, singer of the group Jane's Addiction. Question: Given a reference text about Lollapalooza, where does it take place, who started it and what is it?"
prompt_template = f"""{prompt}

"""
input_ids = tokenizer(prompt_template, return_tensors="pt").input_ids.cuda()


generation_config = GenerationConfig.from_pretrained("TheBloke/Llama-2-7B-GPTQ")
output = model.generate(inputs=input_ids, generation_config=generation_config)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

/home/lujun/anaconda3/envs/causalLLM/lib/python3.9/site-packages/transformers/generation/utils.py:1392: UserWarning: Input length of input_ids is 264, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


Instruction: Please strictly answer the question based on the context I gave to you. Context Information: Lollapalooza /ˌlɒləpəˈluːzə/ (Lolla) is an annual American four-day music festival held in Grant Park in Chicago. It originally started as a touring event in 1991, but several years later, Chicago became its permanent location. Music genres include but are not limited to alternative rock, heavy metal, punk rock, hip hop, and electronic dance music. Lollapalooza has also featured visual arts, nonprofit organizations, and political organizations. The festival, held in Grant Park, hosts an estimated 400,000 people each July and sells out annually. Lollapalooza is one of the largest and most iconic music festivals in the world and one of the longest-running in the United States. Lollapalooza was conceived and created in 1991 as a farewell tour by Perry Farrell, singer of the group Jane's Addiction. Question: Given a reference text about Lollapalooza, where does it take place, who start

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import GenerationConfig


model_name_or_path = "TheBloke/Llama-2-7B-GPTQ"
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path, device_map="auto", trust_remote_code=True, revision="main"
)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Tell me about AI"
prompt_template = f"""{prompt}

"""
input_ids = tokenizer(prompt_template, return_tensors="pt").input_ids.cuda()


# generation_config = GenerationConfig.from_pretrained("TheBloke/Llama-2-7B-GPTQ")
# generation_config = GenerationConfig.from_pretrained(
#     "./saved_model_config/TheBloke-Llama-2-7B-GPTQ/", "config.json"
# )

In [27]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Result: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

Comment: Of course! If you enjoyed "Breaking Bad" and "Band of Brothers," here are some other shows you might enjoy:

1. "The Wire" - This critically acclaimed series explores the drug trade in Baltimore from multiple perspectives, including law enforcement, drug dealers, and politicians.
2. "Narcos" - This Netflix original series tells the true story of Pablo Escobar and the Medellín cartel's rise to power and eventual downfall.
3. "Sons of Anarchy" - This FX series follows the lives of a close-knit outlaw motorcycle club in California, exploring themes of loyalty, betrayal, and redemption.
4. "The Sopranos"


In [28]:
sequences = pipeline(
    "Instruction: Please strictly answer the question based on the context I gave to you.\nContext Information: Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.\n Question: When did Virgin Australia start operating?",
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: Instruction: Please strictly answer the question based on the context I gave to you.
Context Information: Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.
 Question: When did Virgin Australia start operating?
A) 1999
B) 2000
C) 2001
D) 2002

Please select one of the options from the table above.


## Bitsandbytes

## Activation-aware Weight Quantization (AWQ)

## Additional: AQLM